# Neural Machine Translation


> ## Introduction
This lab focuses on using an algorithm based upon Neural Machine Translation to translate sentences from English to French.
> ## Algorithm Description
A Neural Machine Translation system is a neural network that directly models the conditional probability p(y|x) of translating a source sentence, x1, . . .xn, to a target sentence, y1, . . . , ym [[1](https://arxiv.org/abs/1508.04025)]. A basic form of NMT consists of two components:
1. An encoder which computes a representation ***s*** for each source sentence. (We will use an RNN with GRU as hiddent unit.)
2. A decoder which generates one target word at a time and hence de-composes the conditional probability as: $\log p(y|x) =\sum_{j=1}^{m} \log p(y_j|y_{<j},s)$
 
> The Decoder we will use is a non-stacked unidirectional RNN. It is a neural language model conditioned not only on the previously generated target words but also on the source sentence. More precisely, it generates the target sentence: $ y = (y 1 , . . . , y T_y)$ one word $y_t$ at a time based on the distribution: $ P[y_t |{y_1 , ..., y_{t−1}},c_t] = softmax(W_sh̃_t)$, where 
$h̃_t$ , the attentional hidden state, is computed as (biases are not shown for simplicity): $h̃_t = \tanh( W_{c}[c_{t};h_t])$. $h_t$ is the $t^{th}$ hidden state of the decoder, $c_t$ is the source context vector, and $;$ denotes concatenation. $W_s$ and $W_c$ are matrices of trainable parameters.

> **Note:** While all the inputs of the encoder (i.e., all the words of the input sentence) are known at encoding time, the decoder generates one target word at a time, and uses as input at time $t$ its prediction from time $t − 1$.
> ## Global Attention Mechanism
The context vector $c_t$ is computed as a weighted sum of the encoder’s hidden states $h̄_i$ . The vector of weights $\alpha_t$ is obtained by applying a softmax to the output of an alignment operation (score())
between the current target hidden state $h_t$ and all source hidden states $h̄_{i}$'s. $\alpha_t$ indicates which words
in the source sentence are the most likely to help in predicting the next word. *score()* can in theory
be any comparison function. In our implementation, we will use the concat attention formulation of
[[1 Section 3.1](https://arxiv.org/abs/1508.04025)]. An overview is provided in the following figure: 


![Figure-1](https://drive.google.com/uc?export=view&id=12duBYnzyMR06sMv2VfXZWbvK8fvJs7om)



In [0]:
##### Mounting Drive for datasets #######
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#####Creating Encoder Class#########

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.nn.utils.rnn import pad_sequence

from tqdm import tqdm

from nltk import word_tokenize


class Encoder(nn.Module):
    '''
    to be passed the entire source sequence at once
    we use padding_idx in nn.Embedding so that the padding vector does not take gradient (always zero)
    https://pytorch.org/docs/stable/nn.html#gru
    '''
    def __init__(self, vocab_size, embedding_dim, hidden_dim, padding_idx):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx)
        self.rnn = nn.GRU(embedding_dim, hidden_dim)
    
    def forward(self, input):
        print('Start Encoder')
        # fill the gaps # (transform input into embeddings and pass embeddings to RNN)
        embedded_input = self.embedding(input) ## Output: (∗,H), where * is the input shape and H=embedding_dim 
        #Pass input of shape (seq_len, batch, input_size) to GRU
        #Get 1. output of shape (seq_len, batch, num_directions * hidden_size): tensor containing the output features h_t from the last layer of the GRU, for each t. 
        #Get 2. h_n of shape (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t = seq_len
        output_tensor, hs = self.rnn(embedded_input)
        # you should return a tensor of shape (seq,batch,feat)
        print('End Encoder')
        return output_tensor

In [0]:
#####Creating Decoder Class#######
class Decoder(nn.Module):
    '''to be used one timestep at a time
       see https://pytorch.org/docs/stable/nn.html#gru'''
    def __init__(self, vocab_size, embedding_dim, hidden_dim, padding_idx):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx)
        self.rnn = nn.GRU(embedding_dim, hidden_dim)
        self.ff_concat = nn.Linear(2*hidden_dim,hidden_dim) # 60 , 30
        self.predict = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, input, source_context, h):
        # fill the gaps #
        # transform input into embeddings, pass embeddings to RNN, concatenate with source_context and apply tanh, and make the prediction
        # prediction should be of shape (1,batch,vocab), h and tilde_h of shape (1,batch,feat)
        print('Start decoder')
        #print('Shape of input: ',input.size())
        embedded_input=self.embedding(input)
        rnn_output, decoder_hidden = self.rnn(embedded_input,h)
        #print('Shape of rnn output: ',rnn_output.size())
        #print('Shape of source_context: ',source_context.size())
        #rnn_output = rnn_output.squeeze(0)
        #source_context=source_context.squeeze(0)
        #print('Shape of rnn_output after squeeze: ',rnn_output.size())
        concat_input = torch.cat([source_context,decoder_hidden],dim=2)
        #print('Shape after concatenating rnn_output with src_context: ',concat_input.size())
        #print('Next 1')
        output = torch.tanh(self.ff_concat(concat_input))
        output = self.predict(output)
        print('End Decoder')
        return output, decoder_hidden

In [0]:
##### Creating Attention Class #########
 class seq2seqAtt(nn.Module):   
    '''
    concat global attention a la Luong et al. 2015 (subsection 3.1)
    https://arxiv.org/pdf/1508.04025.pdf
    '''
    def __init__(self, hidden_dim, hidden_dim_s, hidden_dim_t):
        super(seq2seqAtt, self).__init__()
        self.ff_concat = nn.Linear(hidden_dim_s+hidden_dim_t,hidden_dim)
        self.ff_score = nn.Linear(hidden_dim,1,bias=False) # just a dot product here
        self.va=nn.Parameter(torch.FloatTensor(hidden_dim))
    
    def forward(self,target_h,source_hs):
        # Create a copy of target_h Tx times ( hidden_dim_s )
        print('Start attention')
        #print('Shape of source_hs: ', source_hs.size())
        target_h_rep = target_h.repeat(source_hs.size(0),1,1) # (1,batch,feat) -> (seq,batch,feat)
        # fill the gaps #
        # implement the score computation part of the concat formulation (see section 3.1. of Luong 2015)
       
        # Concatenate target_h with each source hidden state (h of t')
        concat_output =(self.ff_concat(torch.cat([target_h_rep,source_hs],dim=2))) 
        #print('Concat_ouput shape: ',concat_output.size())
        scores = self.ff_score(torch.tanh(concat_output))  # should be of shape (seq,batch,1)
        #print('Shape of scores: ',scores.size())
        scores = scores.squeeze(dim=2) # (seq,batch,1) -> (seq,batch). dim=2 because we don't want to squeeze the batch dim if batch size = 1
        #print('Shape of scores after squeeze: ',scores.size())
        norm_scores = torch.softmax(scores,0) #alphas
        # Dot product of context vector with source hidden state vector
        source_hs_p = source_hs.permute((2,0,1)) # (seq,batch,feat) -> (feat,seq,batch)
        #print('Shape of norm_scores: ',norm_scores.size())
        #print('Shape of source_hs_p: ',source_hs_p.size())
        weighted_source_hs = (norm_scores * source_hs_p) # (seq,batch) * (feat,seq,batch) (* checks from right to left that the dimensions match)
        ct = torch.sum(weighted_source_hs.permute((1,2,0)),0,keepdim=True) # (feat,seq,batch) -> (seq,batch,feat) -> (1,batch,feat); keepdim otherwise sum squeezes 
        print('Done attention')
        return ct,norm_scores

In [0]:
######Defining Seq2SeqModel Class ########

class seq2seqModel(nn.Module):
    '''the full seq2seq model'''
    ARGS = ['vocab_s','source_language','vocab_t_inv','embedding_dim_s','embedding_dim_t',
     'hidden_dim_s','hidden_dim_t','hidden_dim_att','do_att','padding_token',
     'oov_token','sos_token','eos_token','max_size']
    def __init__(self, vocab_s, source_language, vocab_t_inv, embedding_dim_s, embedding_dim_t, 
                 hidden_dim_s, hidden_dim_t, hidden_dim_att, do_att, padding_token,
                 oov_token, sos_token, eos_token, max_size):
        super(seq2seqModel, self).__init__()
        self.vocab_s = vocab_s
        self.source_language = source_language
        self.vocab_t_inv = vocab_t_inv
        self.embedding_dim_s = embedding_dim_s
        self.embedding_dim_t = embedding_dim_t
        self.hidden_dim_s = hidden_dim_s
        self.hidden_dim_t = hidden_dim_t
        self.hidden_dim_att = hidden_dim_att
        self.do_att = do_att # should attention be used?
        self.padding_token = padding_token
        self.oov_token = oov_token
        self.sos_token = sos_token
        self.eos_token = eos_token
        self.max_size = max_size
        
        self.max_source_idx = max(list(vocab_s.values()))
        #print('max source index',self.max_source_idx)
        #print('source vocab size',len(vocab_s))
        
        self.max_target_idx = max([int(elt) for elt in list(vocab_t_inv.keys())])
        #print('max target index',self.max_target_idx)
        #print('target vocab size',len(vocab_t_inv))
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.encoder = Encoder(self.max_source_idx+1,self.embedding_dim_s,self.hidden_dim_s,self.padding_token).to(self.device)
        self.decoder = Decoder(self.max_target_idx+1,self.embedding_dim_t,self.hidden_dim_t,self.padding_token).to(self.device)
        
        if self.do_att:
            self.att_mech = seq2seqAtt(self.hidden_dim_att,self.hidden_dim_s,self.hidden_dim_t).to(self.device)
    
    def my_pad(self,my_list):
        '''my_list is a list of tuples of the form [(tensor_s_1,tensor_t_1),...,(tensor_s_batch,tensor_t_batch)]
        the <eos> token is appended to each sequence before padding
        https://pytorch.org/docs/stable/nn.html#torch.nn.utils.rnn.pad_sequence'''
        batch_source = pad_sequence([torch.cat((elt[0],torch.LongTensor([self.eos_token]))) for elt in my_list],batch_first=True,padding_value=self.padding_token)
        batch_target = pad_sequence([torch.cat((elt[1],torch.LongTensor([self.eos_token]))) for elt in my_list],batch_first=True,padding_value=self.padding_token)
        return batch_source,batch_target
    
    def forward(self,input,max_size,is_prod):
        #print('In forward')
        if is_prod: 
            input = input.unsqueeze(1) # (seq) -> (seq,1) 1D input <=> we receive just one sentence as input (predict/production mode)
        
        current_batch_size = input.size(1)
    
        # fill the gap #
        # use the encoder
        # Arguments: vocab_size, embedding_dim, hidden_dim, padding_idx
        #encoder = Encoder(len(self.vocab_s),self.embedding_dim_s,self.hidden_dim_s,None)
        source_hs = self.encoder(input)
        
        # = = = decoder part (one timestep at a time)  = = =
        
        target_h = torch.zeros(size=(1,current_batch_size,self.hidden_dim_t)).to(self.device) # init (1,batch,feat)
        
        # fill the gap #
        # (initialize target_input with the proper token)
        target_input = torch.LongTensor([self.sos_token]).repeat(current_batch_size).unsqueeze(0).to(self.device) # init (1,batch)
        print('Shape of target input:',target_input.shape)
        
        pos = 0
        eos_counter = 0
        logits = []
        att_list = []
        while True:
            
            #print(counter)
            #counter = counter + 1
            if self.do_att:
                source_context, attentions = self.att_mech(target_h,source_hs) # (1,batch,feat)
            else:
                source_context = source_hs[-1,:,:].unsqueeze(0) # (1,batch,feat) last hidden state of encoder
            
            # fill the gap #
            # use the decoder
            # Arguments: vocab_size, embedding_dim, hidden_dim, padding_idx)
            #Decoder(len(self.vocab_t_inv),self.embedding_dim_t,self.hidden_dim_t,None)
            prediction, target_h = self.decoder(target_input,source_context,target_h)
            
            logits.append(prediction) # (1,batch,vocab)
            att_list.append(attentions.cpu().detach().numpy().squeeze())

            
            # fill the gap #
            # get the next input to pass the decoder
            #source_context = self.att_mech.forward(target_h,source_hs)
            #topv, topi = prediction.topk(1)
            target_input = torch.argmax(prediction,2)#topi.squeeze(-1).detach().unsqueeze(0)

           
            #target_input = target_input.unsqueeze(0)
            #Variable(torch.LongTensor([self.SOS] * batch_size)).squeeze(-1)
            
            eos_counter += torch.sum(target_input==self.eos_token).item()
            
            pos += 1
            if pos>=max_size or (eos_counter == current_batch_size and is_prod):
                break
        
        to_return = torch.cat(logits,0) # logits is a list of tensors -> (seq,batch,vocab)
        
        if is_prod:
            to_return = to_return.squeeze(dim=1) # (seq,vocab)
        
        return to_return
    
    def fit(self, trainingDataset, testDataset, lr, batch_size, n_epochs, patience):
        
        parameters = [p for p in self.parameters() if p.requires_grad]
        
        optimizer = optim.Adam(parameters, lr=lr)
        
        criterion = torch.nn.CrossEntropyLoss(ignore_index=self.padding_token) # the softmax is inside the loss!
        
        # https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
        # we pass a collate function to perform padding on the fly, within each batch
        # this is better than truncation/padding at the dataset level
        train_loader = data.DataLoader(trainingDataset, batch_size=batch_size, 
                                       shuffle=True, collate_fn=self.my_pad) # returns (batch,seq)
        
        test_loader = data.DataLoader(testDataset, batch_size=512,
                                      collate_fn=self.my_pad)
        
        tdqm_dict_keys = ['loss', 'test loss']
        tdqm_dict = dict(zip(tdqm_dict_keys,[0.0,0.0]))
        
        patience_counter = 1
        patience_loss = 99999
        
        for epoch in range(n_epochs):
                   
            with tqdm(total=len(train_loader),unit_scale=True,postfix={'loss':0.0,'test loss':0.0},
                      desc="Epoch : %i/%i" % (epoch, n_epochs-1),ncols=100) as pbar:
                for loader_idx, loader in enumerate([train_loader, test_loader]):
                    total_loss = 0
                    # set model mode (https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
                    if loader_idx == 0:
                        self.train()
                    else:
                        self.eval()
                    for i, (batch_source,batch_target) in enumerate(loader):
                        batch_source = batch_source.transpose(1,0).to(self.device) # RNN needs (seq,batch,feat) but loader returns (batch,seq)                        
                        batch_target = batch_target.transpose(1,0).to(self.device) # (seq,batch)
                        
                        # are we using the model in production / as an API?
                        is_prod = len(batch_source.shape)==1 # if False, 2D input (seq,batch), i.e., train or test
                        
                        if is_prod:
                            max_size = self.max_size
                            self.eval()
                        else:
                            max_size = batch_target.size(0) # no need to continue generating after we've exceeded the length of the longest ground truth sequence
                        print('Start fit')
                        unnormalized_logits = self.forward(batch_source,max_size,is_prod)
                        print('Shape of logits: ',unnormalized_logits.shape)
                        print('End fit')
                        sentence_loss = criterion(unnormalized_logits.flatten(end_dim=1),batch_target.flatten())
                  
                        total_loss += sentence_loss.item()
                        
                        tdqm_dict[tdqm_dict_keys[loader_idx]] = total_loss/(i+1)
                        
                        pbar.set_postfix(tdqm_dict)
                        
                        if loader_idx == 0:
                            optimizer.zero_grad() # flush gradient attributes
                            sentence_loss.backward() # compute gradients
                            optimizer.step() # update
                            pbar.update(1)
            
            if total_loss > patience_loss:
                patience_counter += 1
            else:
                patience_loss = total_loss
                patience_counter = 1 # reset
            
            if patience_counter>patience:
                break
    
    def sourceNl_to_ints(self,source_nl):
        '''converts natural language source sentence into source integers'''
        source_nl_clean = source_nl.lower().replace("'",' ').replace('-',' ')
        source_nl_clean_tok = word_tokenize(source_nl_clean,self.source_language)
        source_ints = [int(self.vocab_s[elt]) if elt in self.vocab_s else \
                       self.oov_token for elt in source_nl_clean_tok]
        
        source_ints = torch.LongTensor(source_ints).to(self.device)
        return source_ints 
    
    def targetInts_to_nl(self,target_ints):
        '''converts integer target sentence into target natural language'''
        return ['<PAD>' if elt==self.padding_token else '<OOV>' if elt==self.oov_token \
                else '<EOS>' if elt==self.eos_token else '<SOS>' if elt==self.sos_token\
                else self.vocab_t_inv[elt] for elt in target_ints]
    
    def predict(self,source_nl):
        source_ints = self.sourceNl_to_ints(source_nl)
        logits,attentions = self.forward(source_ints,self.max_size,True) # (seq) -> (<=max_size,vocab)
        target_ints = logits.argmax(-1).squeeze() # (<=max_size,1) -> (<=max_size)
        target_nl = self.targetInts_to_nl(target_ints.tolist())
        return ' '.join(target_nl), attentions
        
    def save(self,path_to_file):
        attrs = {attr:getattr(self,attr) for attr in self.ARGS}
        attrs['state_dict'] = self.state_dict()
        torch.save(attrs,path_to_file)
    
    @classmethod # a class method does not see the inside of the class (a static method does not take self as first argument)
    def load(cls,path_to_file):
        attrs = torch.load(path_to_file, map_location=lambda storage, loc: storage) # allows loading on CPU a model trained on GPU, see https://discuss.pytorch.org/t/on-a-cpu-device-how-to-load-checkpoint-saved-on-gpu-device/349/6
        state_dict = attrs.pop('state_dict')
        new = cls(**attrs) # * list and ** names (dict) see args and kwargs
        new.load_state_dict(state_dict)
        return new

    def showAttention(self,input_sentence, output_words, attentions, path_to_output_images):
       # Set up figure with colorbar
        #figure(num=None, figsize=(20, 20), dpi=80, facecolor='w', edgecolor='k')
        fig = plt.figure(figsize = (10,10))
        ax = fig.add_subplot(111)
        cax = ax.matshow(np.array(attentions,dtype=np.float64), cmap='bone')
        fig.colorbar(cax)

        # Set up axes
        ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
        ax.set_yticklabels([''] + output_words )

        # Show label at every tick
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.plot()
        
        fig.savefig(path_to_output_images + 'attention_evaluation.png')


    def evaluateAndShowAttention(self,input_sentence, path_to_output_images):
      output_words, attentions = self.predict(input_sentence)
      # print('input =', input_sentence)
      # print('output =', output_words)
      output_words = output_words.split(' ')
      for i,elt in enumerate(output_words):
          # print(elt)
          if elt=='.':
              output_words = output_words[:i]
              attentions = attentions[:i]
              break
      self.showAttention(input_sentence,output_words, attentions, path_to_output_images)


In [0]:
###Test Phase#####

import sys
import json

import torch
from torch.utils import data

# = = = = = = = = = = =

path_root = '/content/drive/My Drive/Colab Notebooks/AlteGrad2019/Lab4/'

#path_to_model = path_root + '/code/'
#sys.path.insert(0, path_to_model)

#from model import seq2seqModel

path_to_data = path_root + 'data/'
path_to_save_models = path_root + 'models/'

# = = = = = = = = = = =

class Dataset(data.Dataset):
  def __init__(self, pairs):
        self.pairs = pairs

  def __len__(self):
        return len(self.pairs) # total nb of observations

  def __getitem__(self, idx):
        source, target = self.pairs[idx] # one observation
        return torch.LongTensor(source), torch.LongTensor(target)


def load_pairs(train_or_test):
    with open(path_to_data + 'pairs_' + train_or_test + '_ints.txt', 'r', encoding='utf-8') as file:
        pairs_tmp = file.read().splitlines()
    pairs_tmp = [elt.split('\t') for elt in pairs_tmp]
    pairs_tmp = [[[int(eltt) for eltt in elt[0].split()],[int(eltt) for eltt in \
                  elt[1].split()]] for elt in pairs_tmp]
    return pairs_tmp


# = = = = = = = = = = =

do_att = True # should always be set to True
is_prod = True # production mode or not

if not is_prod:
        
    pairs_train = load_pairs('train')
    pairs_test = load_pairs('test')
    
    with open(path_to_data + 'vocab_source.json','r') as file:
        vocab_source = json.load(file) # word -> index
    
    with open(path_to_data + 'vocab_target.json','r') as file:
        vocab_target = json.load(file) # word -> index
    
    vocab_target_inv = {v:k for k,v in vocab_target.items()} # index -> word
    
    print('data loaded')
        
    training_set = Dataset(pairs_train)
    test_set = Dataset(pairs_test)
    
    print('data prepared')
    
    print('= = = attention-based model?:',str(do_att),'= = =')
    
    model = seq2seqModel(vocab_s=vocab_source,
                         source_language='english',
                         vocab_t_inv=vocab_target_inv,
                         embedding_dim_s=40,
                         embedding_dim_t=40,
                         hidden_dim_s=30,
                         hidden_dim_t=30,
                         hidden_dim_att=20,
                         do_att=do_att,
                         padding_token=0,
                         oov_token=1,
                         sos_token=2,
                         eos_token=3,
                         max_size=30) # max size of generated sentence in prediction mode
    
    model.fit(training_set,test_set,lr=0.001,batch_size=64,n_epochs=20,patience=2)
    model.save(path_to_save_models + 'my_model.pt')

else:
    
    model = seq2seqModel.load(path_to_save_models + 'pretrained_moodle.pt')
    
    to_test = ['I am a student.',
               'I have a red car.',  # inversion captured
               'I love playing video games.',
                'This river is full of fish.', # plein vs pleine (accord)
                'The fridge is full of food.',
               'The cat fell asleep on the mat.',
               'my brother likes pizza.', # pizza is translated to 'la pizza'
               'I did not mean to hurt you', # translation of mean in context
               'She is so mean',
               'Help me pick out a tie to go with this suit!', # right translation
               "I can't help but smoking weed", # this one and below: hallucination
               'The kids were playing hide and seek',
               'The cat fell asleep in front of the fireplace']
    
    for elt in to_test:
        print('= = = = = \n','%s -> %s' % (elt,model.predict(elt)))

RuntimeError: ignored